<a href="https://colab.research.google.com/github/lunaB/tutor_resource/blob/main/20%ED%9A%8C%EC%B0%A8_%EA%B3%BC%EC%99%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 24.1MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.51MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 23.7MB/s]


In [5]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('train.csv')
train_df = train_df[['Survived','Pclass','Sex','Fare']]

test_df = pd.read_csv('test.csv')
test_df = test_df[['Pclass','Sex','Fare']]

In [6]:
train_df.loc[:,'Sex'] = train_df.loc[:,'Sex'].str.replace('female', '0').replace('male', '1')
train_df['Sex'] = train_df['Sex'].astype(np.int32)
train_df[['Pclass','Sex','Fare']].values

array([[ 3.    ,  1.    ,  7.25  ],
       [ 1.    ,  0.    , 71.2833],
       [ 3.    ,  0.    ,  7.925 ],
       ...,
       [ 3.    ,  0.    , 23.45  ],
       [ 1.    ,  1.    , 30.    ],
       [ 3.    ,  1.    ,  7.75  ]])

In [21]:
import torch 
import torch.nn as nn
import torch.optim as optim

x_train = torch.FloatTensor(train_df[['Pclass','Sex','Fare']].values)
y_train = torch.LongTensor(train_df['Survived'].values)

model = nn.Linear(3, 2)

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss = nn.CrossEntropyLoss()

for i in range(100):
    y = model(x_train)
    
    cost = loss(y, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if i%10 == 0:
        print(i, cost.item(), '%.2f%%'%((model(x_train).argmax(1) == y_train).float().mean()*100))


0 14.866499900817871 39.84%
10 11.609554290771484 39.84%
20 8.366689682006836 39.84%
30 5.243552207946777 40.07%
40 2.683316230773926 62.07%
50 1.143168330192566 68.13%
60 1.0604337453842163 62.74%
70 0.8842207193374634 68.91%
80 0.8626347184181213 68.46%
90 0.7853472828865051 68.69%


In [24]:
W = torch.zeros((3, 2))
W
b = torch.zeros(2)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [25]:
a = nn.Linear(3,2)
a

Linear(in_features=3, out_features=2, bias=True)

In [38]:
softmax_y = torch.functional.F.softmax(torch.FloatTensor([[0.1,0.2,0.9,0.8],[0.1,0.2,0.9,0.8]]), dim=1)

In [40]:
softmax_y.argmax(dim=1)

tensor([2, 2])

In [74]:
dictionary = ['토끼', '뱀', '고양이']

In [81]:
X = torch.FloatTensor([[0.2,0.3,0.9],
                       [0.5,0.7,0.9]])

Yhat = torch.LongTensor([1,0])

W = torch.randn((3,3), requires_grad=True)
b = torch.randn((3,), requires_grad=True)

Y = X.matmul(W)+b
Y

tensor([[1.8255, 2.0213, 0.3032],
        [1.7722, 3.1658, 0.7909]], grad_fn=<AddBackward0>)

In [66]:
softmax_y = torch.functional.F.softmax(Y, dim=1)
softmax_y

tensor([[0.4968, 0.0439, 0.4594],
        [0.2550, 0.0244, 0.7207]])

In [76]:
ans = softmax_y.argmax(dim=1)
print(ans)
print(dictionary[ans[0]])
print(dictionary[ans[1]])

tensor([0, 2])
토끼
고양이


In [73]:
(ans == Yhat).float().mean()

tensor(0.)

In [79]:
error = torch.functional.F.cross_entropy(Y, Yhat)

tensor(2.2466)

In [89]:
# 방법1
X = torch.FloatTensor([[0.2,0.3,0.9],
                       [0.5,0.7,0.9]])

Yhat = torch.LongTensor([1,0])

W = torch.randn((3,3), requires_grad=True)
b = torch.randn((3,), requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.01)

for i in range(5000):
    Y = X.matmul(W)+b
    softmax_y = torch.functional.F.softmax(Y, dim=1)
    error = torch.functional.F.cross_entropy(Y, Yhat)

    if i%500 == 0:
        print(error.item(), (softmax_y.argmax(dim=1) == Yhat).float().mean())

    optimizer.zero_grad()
    error.backward()
    optimizer.step()

1.6411867141723633 tensor(0.)
0.595460057258606 tensor(1.)
0.47965008020401 tensor(1.)
0.4072633385658264 tensor(1.)
0.3533758521080017 tensor(1.)
0.31114768981933594 tensor(1.)
0.27715209126472473 tensor(1.)
0.2492634803056717 tensor(1.)
0.22603845596313477 tensor(1.)
0.20644986629486084 tensor(1.)


In [94]:
# 방법2
X = torch.FloatTensor([[0.2,0.3,0.9],
                       [0.5,0.7,0.9]])

Yhat = torch.LongTensor([1,0])

model = nn.Linear(3,2)
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss = nn.CrossEntropyLoss()

for i in range(5000):

    Y = model(X)
    softmax_y = torch.functional.F.softmax(Y, dim=1)
    cost = loss(Y, Yhat)
    if i%500 == 0:
        print(cost.item(), (softmax_y.argmax(dim=1) == Yhat).float().mean())

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

0.6907826066017151 tensor(0.5000)
0.5619001388549805 tensor(1.)
0.47742143273353577 tensor(1.)
0.41116052865982056 tensor(1.)
0.3585607409477234 tensor(1.)
0.3162539303302765 tensor(1.)
0.2817738950252533 tensor(1.)
0.25331348180770874 tensor(1.)
0.2295401692390442 tensor(1.)
0.20946243405342102 tensor(1.)
